# **7. SIMILARITY MEASURES AND DUPLICATE DETECTION**

**PART 1: SIMILARITIES MEASURES**

- Distance-Based Comparison Functions

  - String-Based Distance Functions
    - Edit-based Similarity Measures
      - Edit Distance/Levenshtein Distance
    - Phonetic-based Algorithms
      - Sounded Code (American Soundex)
    - Jaro-Winkler String Comparator
  - Item-Based Distance Functions
    - Jaccard Distance
    - TF-IDF: Term Frequency - Inverse Document Frequency (or Cosine Similarity)

Import libraries:

In [6]:
!pip install python-Levenshtein pyphonetics jaro-winkler recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.7 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np

import Levenshtein as lev
import pyphonetics
from pyphonetics import Soundex
import jaro
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import recordlinkage
from recordlinkage.datasets import load_febrl1

**1. Edit-based** *Edit Distance/Levenstein Distance*

  Minimum number of edits from one word to the other.

        Similarity = 1 - Levenshtein Distance/Max Length
        Levenstein Ratio = Levenstein Similarity/Alignment Length

In [3]:
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1

    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1,
               LD(s[:-1], t[:-1]) + cost])
    return res

In [14]:
#Distance computation
name1 = "Maria"
name2 = "Mariella"
L = LD(name1,name2)
L

3

In [15]:
#Maximum length
ML = max(len(name1),len(name2))
ML

8

In [16]:
#Similarity evaluation = 1 - Levenshtein Distance/Max Length
S = 1-L/ML
S

0.625

In [17]:
#There is also a library
L = lev.distance(name1,name2)
L

3

In [18]:
#Compute the lev ratio = Levenshtein Similarity/Alignment Length
L = lev.ratio(name1,name2)
L

0.7692307692307692

**2. Phonetic-based** *American Soundex Algorithm*

These algorithms can find similar sounding terms and names.

      1. The first character of the word is retained as the first character of the Soundex code.
      2. The following letters are discarded: a,e,i,o,u,h,w, and y.
      3. Remaining consonants are given a code number.
      4. If consonants having the same code number appear consecutively, the number will only be coded once. (e.g. "B233" becomes "B23")
      5. The resulting code is modified so that it becomes exactly four characters long: if it is less than 4 characters, zeroes are added to the end (e.g. "B2" becomes "B200")
      6. If it is more than 4 characters, the code is truncated (e.g. "B2435" becomes "B243")


In [ ]:
#help(pyphonetics)

In [19]:
soundex = Soundex()

In [21]:
#Sounded code of Maria
soundex.phonetics(name1)

'M600'

In [22]:
#Sounded code of Mariella
soundex.phonetics(name2)

'M640'

In [23]:
soundex.sounds_like(name1,name2) #(binary return, True only if are the same)

False

In [25]:
#Compute the Levenshtein distance between Soundex Codes
soundex.distance(name1,name2,metric='levenshtein')


1

**3. Jaro-Winkler** *Similarity Index*

The Jaro-Winkler string comparator counts:

  - the number c of matching characters between two strings
  (limited to the greatest integer of half the length of the longer string --> c <= max(len(s1),len(s2))/2 )
  - the number of transpositions t that are the number of matching characters that are not in the right order divided by two
  - n is the length of string 1
  - m is the length of string 2

        Jaro-Similarity = 1/3 * (c/m + c/n + (c-t)/c)

        Jaro–Winkler Similarity = sim(j) + lp(1 - sim(j)) where:
          -  sim(j) is the Jaro-Similarity for strings s1 and s2
          - l is the length of common prefix at the start of the string up to a maximum of 4 characters
          - p is a constant scaling factor for how much the score is adjusted upwards for having common prefixes

In [26]:
jaro.jaro_winkler_metric(name1,name2)

0.925

In [ ]:
#help(jaro)

**4. Item-based** *Jaccard Similarity*

Jaccard similarity index is defined as the intersection of the two sets of words dividing by the union of them.

- Let A and B the set of words in S1 and S2 then:

      Jaccard (S1, S2) = |A ∩ B| / |A U B|

In [27]:
a = [0, 1, 2, 5, 6, 8, 9]
b = [0, 2, 3, 4, 5, 7, 9]

In [28]:
#Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [29]:
#Compute Jaccard Similarity
jaccard(a,b)

0.4

In [30]:
#Jaccard Similarity between two sets of words
words_doc1 = {'data', 'are', 'valuable', 'assets', 'of', 'the', 'company'}
words_doc2 = {'data', 'are', 'assets'}

In [31]:
#Compute Jaccard Similarity
jaccard(words_doc1,words_doc2)

0.42857142857142855

**5. Item-based** *Text Similarity (or Cosine Similarity)*

**TF-IDF**: *Term Frequency - Inverse Document Frequency*

The idea is to assign higher weights to words appearing frequently in a document (*TF weight*) and then to assign lower weights to words that appear frequently in the whole set of documents (*IDF weight*).

      1. We obtain vectors of weights (e.g., U and V for different documents)
      2. We then calculate the similarity using cosine similarity with these vectors.

In [4]:
corpus = ['Data are valuable assets of the company',
          'Data quality is important for a company',
          'Data are assets',
          'Data Quality is important for data scientists',
          'Data are valuable products of the company']

In [8]:
#Initialize an instance of count Vectorizer
count_vectorizer = CountVectorizer()
# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [9]:
#Generate the count matrix and the tf-idf vectors for the corpus
count_matrix = count_vectorizer.fit_transform(corpus)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [10]:
#Extract the words
count_tokens = count_vectorizer.get_feature_names_out()
tfidf_tokens = tfidf_vectorizer.get_feature_names_out()

In [11]:
#Create the dataframe with the count matrix and the tf-idf vectors
df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),index = ['Doc1','Doc2','Doc3','Doc4','Doc5'],columns = tfidf_tokens)
df_countvect = pd.DataFrame(data = count_matrix.toarray(),index = ['Doc1','Doc2','Doc3','Doc4','Doc5'],columns = count_tokens)

In [12]:
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      are  assets  company  data  for  important  is  of  products  quality  \
Doc1    1       1        1     1    0          0   0   1         0        0   
Doc2    0       0        1     1    1          1   1   0         0        1   
Doc3    1       1        0     1    0          0   0   0         0        0   
Doc4    0       0        0     2    1          1   1   0         0        1   
Doc5    1       0        1     1    0          0   0   1         1        0   

      scientists  the  valuable  
Doc1           0    1         1  
Doc2           0    0         0  
Doc3           0    0         0  
Doc4           1    0         0  
Doc5           0    1         1  

TD-IDF Vectorizer

           are    assets   company      data       for  important        is  \
Doc1  0.346868  0.417868  0.346868  0.246800  0.000000   0.000000  0.000000   
Doc2  0.000000  0.000000  0.369830  0.263137  0.445529   0.445529  0.445529   
Doc3  0.581482  0.700505  0.000000  0.413729  

In [43]:
#Compute and print the cosine similarity between the documents
pd.DataFrame(cosine_similarity(df_tfidfvect, df_tfidfvect),columns=['Doc1','Doc2','Doc3','Doc4','Doc5'],index=['Doc1','Doc2','Doc3','Doc4','Doc5'])
#

,Doc1,Doc2,Doc3,Doc4,Doc5
Doc1,1.000000,0.193224,0.596524,0.110729,0.789258
Doc2,0.193224,1.000000,0.108867,0.794951,0.184767
Doc3,0.596524,0.108867,1.000000,0.185624,0.290508
Doc4,0.110729,0.794951,0.185624,1.000000,0.105883
Doc5,0.789258,0.184767,0.290508,0.105883,1.000000


**PART 2: DUPLICATE DETECTION**

**Duplicate Detection** is the discovery of multiple representations of the same real-world object.
- Exact Matching
- Record Linkage (Not-Exact Matching)

**1. Exact Matching**

In [44]:
data = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/BEERS.csv")


In [45]:
data.duplicated().sum()

np.int64(9)

In [48]:
print(data[data.duplicated()])

        abv   ibu    id                          name  \
1582   0.08  35.0     5                      Old Chub   
1606  55.00  55.0  1946   Overgrown American Pale Ale   
1676  57.00  68.0   711        Over the Rail Pale Ale   
1748  62.00   NaN  2373                   Oktoberfest   
1826   0.05  45.0  1324     Schlafly Yakima Wheat Ale   
1882   0.06  48.0   512  Autumnation (2011-12) (2011)   
2291  65.00  33.0  1932          Thai Style White IPA   
2340   0.07   NaN  2222                        10 Ton   
2406  45.00  25.0  1514               Easy Day Kolsch   

                        style brewery_id  ounces  
1582             Scottish Ale        166    12.0  
1606  American Pale Ale (APA)        261    12.0  
1676  American Pale Ale (APA)        135    12.0  
1748     Märzen / Oktoberfest         43    12.0  
1826  American Pale Wheat Ale        428    12.0  
1882              Pumpkin Ale         46    16.0  
2291       American White IPA         51    12.0  
2340            Oatme

In [ ]:
data.drop_duplicates()

**2. Record Linkage (RL)**

*Record Linkage* is the task of finding records in a data set that refer to the same entity (not-exact matching) across different data sources.

- Steps:
      1. Find the candidate records to compare
      2. Compare the candidate records and find duplicates

**RL** *Single Data Source*: Record linkage is a good option to find duplicates within a dataset.

**2.1 Find the candidate pairs**

The goal is to detect the record pairs to compare.

This process of making record pairs is also called ‘indexing’.

The ***recordlinkage*** library builds the set of pairs to compare.

In [18]:
data1 = load_febrl1()
data1

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-188-dup-0,stephanie,geu,28,bainton crescent,masonic memorial village,maryborough,2541,sa,19421008,3997529
rec-334-dup-0,nicholas,NaN,289,britten-jonues drive,jabaru court,paddington,2000,vic,19970422,5062738
rec-469-dup-0,lachlan,katsiavos,29,paul coe cdrescent,NaN,casual,2913,nsw,19380406,4112327


In [16]:
#First, load the recordlinkage.Index class and call the .full method.
#This object generates a full index on a .index(...) call.
#In case of deduplication of a single dataframe, one dataframe is sufficient as input argument.
indexer = recordlinkage.Index()

In [19]:
indexer.full()
candidate_links = indexer.index(data1)


In [20]:
#Numeber of comparison generated = (1000*1000-1000)/2 = 499500
print(len(data1), len(candidate_links))
candidate_links


1000 499500


MultiIndex([( 'rec-122-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-223-org'),
            ('rec-10-dup-0',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-373-org'),
            ( 'rec-227-org',   'rec-223-org'),
            ( 'rec-227-org',   'rec-122-org'),
            ( 'rec-227-org',   'rec-373-org'),
            ( 'rec-227-org',  'rec-10-dup-0'),
            ...
            ( 'rec-212-org',   'rec-205-org'),
            ( 'rec-212-org', 'rec-416-dup-0'),
            ( 'rec-212-org',   'rec-330-org'),
            ( 'rec-212-org',   'rec-385-org'),
            ( 'rec-212-org', 'rec-113-dup-0'),
            ( 'rec-212-org',   'rec-327-org'),
            ( 'rec-212-org', 'rec-188-dup-0'),
            ( 'rec-212-org', 'rec-334-dup-0'),
            ( 'rec-212-org', 'rec-469-dup-0'),
            ( 'rec-212-org', 'rec-350-dup-0')],
           names=['rec_id_1', 'rec_id_2'], 

**2.1.1 Find the candidate pairs** with *Blocking*

The file is partitioned in exclusive blocks and the comparison is limited to records within the same block.

***Blocking*** can be implemented by choosing a blocking key and grouping into a block all records that have the same values on the blocking key.

We cannot compare records from different blocks, this is a *huge limitation*.

In [21]:
#Many of these record pairs do not belong to the same person.
#The recordlinkage toolkit has some more advanced indexing methods to reduce the number of record pairs.
#Obvious not-matches are left out of the index.
#Note that if a matching record pair is not included in the index, it can not be matched anymore
indexer = recordlinkage.Index()
indexer.block('given_name') # the blocks are builded on the basis of the provided attribute
candidate_links = indexer.index(data1)

In [22]:
#The comparison will be limited to records with the same given_name
print(len(candidate_links))


2082


In [23]:
#Surname is discriminating more
indexer = recordlinkage.Index()
indexer.block('surname')
candidate_links = indexer.index(data1)

print(len(candidate_links))
candidate_links

1707


MultiIndex([(  'rec-335-org', 'rec-251-dup-0'),
            (  'rec-251-org', 'rec-251-dup-0'),
            (  'rec-251-org',   'rec-335-org'),
            ('rec-335-dup-0', 'rec-251-dup-0'),
            ('rec-335-dup-0',   'rec-335-org'),
            ('rec-335-dup-0',   'rec-251-org'),
            ('rec-320-dup-0', 'rec-305-dup-0'),
            (  'rec-167-org',   'rec-361-org'),
            ( 'rec-23-dup-0',    'rec-23-org'),
            (  'rec-419-org',   'rec-122-org'),
            ...
            (  'rec-327-org',    'rec-52-org'),
            (  'rec-327-org',   'rec-300-org'),
            (  'rec-327-org',   'rec-193-org'),
            (  'rec-327-org', 'rec-144-dup-0'),
            (  'rec-327-org', 'rec-300-dup-0'),
            (  'rec-327-org',   'rec-205-org'),
            ('rec-350-dup-0',   'rec-350-org'),
            (  'rec-212-org', 'rec-325-dup-0'),
            (  'rec-212-org',   'rec-325-org'),
            (  'rec-212-org', 'rec-212-dup-0')],
           names=['rec_

In [24]:
#You can define the blocks among different attributes
indexer = recordlinkage.Index()
indexer.block('given_name', 'surname')
#candidate_links = indexer.index(data1)

#print(len(candidate_links))
#candidate_links

<Index>

In [25]:
indexer = recordlinkage.Index()
indexer.block(on='given_name')
#candidate_links = indexer.index(data1)
#print(len(candidate_links))
#candidate_links

<Index>

**2.1.2 Find the candidate pairs** with *Sorted Neighborhood*

***Sorted neighborhood*** consists of sorting a file and then moving a window of a fixed size on the file, comparing only records within the window.

In [26]:
#First you have to sort the record in the database on the basis of a key (we use attributes to sort)
#Increasing the length of the window, the number of candidate links increases as well
indexer = recordlinkage.index.SortedNeighbourhood(
        on='given_name', window=9
    )

candidate_links = indexer.index(data1)
print(len(candidate_links))
candidate_links

11447


/tmp/ipython-input-3332702597.py:3: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  indexer = recordlinkage.index.SortedNeighbourhood(


MultiIndex([('rec-190-dup-0',   'rec-373-org'),
            (   'rec-73-org',   'rec-122-org'),
            (  'rec-186-org', 'rec-190-dup-0'),
            (  'rec-262-org',   'rec-452-org'),
            (  'rec-113-org',   'rec-409-org'),
            ('rec-291-dup-0',    'rec-23-org'),
            ('rec-291-dup-0',  'rec-23-dup-0'),
            (  'rec-102-org',   'rec-403-org'),
            (  'rec-312-org', 'rec-254-dup-0'),
            (   'rec-69-org',   'rec-350-org'),
            ...
            ('rec-469-dup-0',   'rec-279-org'),
            ('rec-469-dup-0',  'rec-73-dup-0'),
            ('rec-469-dup-0',   'rec-269-org'),
            ('rec-469-dup-0', 'rec-154-dup-0'),
            ('rec-469-dup-0',   'rec-159-org'),
            ('rec-350-dup-0',    'rec-69-org'),
            ('rec-350-dup-0', 'rec-258-dup-0'),
            ('rec-350-dup-0',  'rec-69-dup-0'),
            ('rec-350-dup-0',  'rec-66-dup-0'),
            ('rec-350-dup-0',    'rec-66-org')],
           names=['rec_

**2.2 Comparison between candidate pairs**

In [27]:
#Compare records
compare_cl = recordlinkage.Compare()
#Define the way in which the tuples should be compared specifying the list of attributes
# these two attributes have to match exactly
compare_cl.exact('given_name', 'given_name', label='given_name')
#Here we compare attributes based on a similarity measure
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')

compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, data1)
features

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-190-dup-0 rec-373-org             0      0.0              0       0   
rec-73-org    rec-122-org             0      0.0              0       0   
rec-186-org   rec-190-dup-0           0      0.0              0       0   
rec-262-org   rec-452-org             0      0.0              0       0   
rec-113-org   rec-409-org             0      0.0              0       0   
...                                 ...      ...            ...     ...   
rec-350-dup-0 rec-69-org              0      0.0              0       0   
              rec-258-dup-0           0      0.0              0       0   
              rec-69-dup-0            0      0.0              0       0   
              rec-66-dup-0            0      0.0              0       0   
              rec-66-org              0      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-190-dup-0 rec-373-org        0        0.0  
rec-73-org    rec-122-org        0        0.0  
rec-186-org   rec-190-dup-0      1        0.0  
rec-262-org   rec-452-org        0        0.0  
rec-113-org   rec-409-org        0        0.0  
...                            ...        ...  
rec-350-dup-0 rec-69-org         0        0.0  
              rec-258-dup-0      0        0.0  
              rec-69-dup-0       0        0.0  
              rec-66-dup-0       0        0.0  
              rec-66-org         0        0.0  

[11447 rows x 6 columns]

In [28]:
#The last step is to decide which records belong to the same person. In this example, we keep it simple:
features.sum(axis=1).value_counts().sort_index(ascending=False)

,count
6.0,142
5.0,167
4.0,53
3.0,15
2.0,388
1.0,3576
0.0,7106


In [30]:
##Compare the matches
matches = features[features.sum(axis=1) > 5]

#print(len(matches))
matches

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-335-dup-0,rec-335-org,1,1.0,1,1,1,1.0
rec-23-dup-0,rec-23-org,1,1.0,1,1,1,1.0
rec-452-dup-0,rec-452-org,1,1.0,1,1,1,1.0
rec-316-org,rec-316-dup-0,1,1.0,1,1,1,1.0
rec-77-dup-0,rec-77-org,1,1.0,1,1,1,1.0
...,...,...,...,...,...,...,...
rec-30-dup-0,rec-30-org,1,1.0,1,1,1,1.0
rec-138-org,rec-138-dup-0,1,1.0,1,1,1,1.0
rec-113-dup-0,rec-113-org,1,1.0,1,1,1,1.0


**RL** *Multiple Data Sources*

In [31]:
hospital_accounts = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/HOSPITAL_ACC.csv")
hospital_reimbursement = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/HOSPITAL_REIM.csv")

1. ***Sorted Neighbourhood*** to find candidate pairs

In [32]:
indexer = recordlinkage.index.SortedNeighbourhood(
    left_on='Facility Name', right_on='Provider Name', window=15
)

candidate_links = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidate_links))
candidate_links

38900


MultiIndex([(   0,  717),
            (   1,  516),
            (   2,  132),
            (   4, 2168),
            (   6, 1430),
            (  10, 2380),
            (  13, 2183),
            (  15, 2086),
            (  17, 1588),
            (  19, 2267),
            ...
            (5319, 2572),
            (5320,  528),
            (5323, 1173),
            (5324, 2010),
            (5325, 2649),
            (5328, 2070),
            (5332, 1777),
            (5333, 1167),
            (5336, 1172),
            (5338, 1256)],
           length=38900)

2. ***Compare*** the founded canditate pairs

In [33]:
compare_cl = recordlinkage.Compare()

compare_cl.exact('Facility Name', 'Provider Name', label='name')
compare_cl.string('Address', 'Provider Street Address', method='jarowinkler', threshold=0.85, label='address')
compare_cl.string('City', 'Provider City', threshold=0.85, label='city')
compare_cl.exact('State', 'Provider State', label='state')
compare_cl.exact('ZIP Code', 'Provider Zip Code', label='address_1')

features = compare_cl.compute(candidate_links, hospital_accounts, hospital_reimbursement)
features

,,name,address,city,state,address_1
0,717,0,0.0,0.0,0,0
1,516,0,0.0,0.0,0,0
2,132,0,0.0,0.0,0,0
4,2168,0,0.0,0.0,0,0
6,1430,0,0.0,0.0,0,0
...,...,...,...,...,...,...
5328,2070,0,0.0,0.0,0,0
5332,1777,0,0.0,0.0,0,0
5333,1167,0,0.0,0.0,0,0
5336,1172,0,0.0,0.0,0,0


In [34]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

,count
5.0,2240
4.0,129
3.0,48
2.0,280
1.0,5442
0.0,30761


In [35]:
matches = features[features.sum(axis=1) > 4]

#print(len(matches))
matches

,,name,address,city,state,address_1
14,1777,1,1.0,1.0,1,1
27,1502,1,1.0,1.0,1,1
28,779,1,1.0,1.0,1,1
34,2170,1,1.0,1.0,1,1
38,1308,1,1.0,1.0,1,1
...,...,...,...,...,...,...
5326,2400,1,1.0,1.0,1,1
5328,65,1,1.0,1.0,1,1
5329,2213,1,1.0,1.0,1,1
5334,988,1,1.0,1.0,1,1


**Summary**

*Edit Distance/Levenshtein Distance*
- Levenshtein.distance()
- Levenshtein.ratio()

*Sounded Code (American Soundex)*
- Soundex().phonetic()
- Soundex().sounds_like()
- Soundex().distance()

*Jaro-Winkler*
- jaro.jaro_winkler_metric


*TF-IDF (sklearn)*
- CountVectorizer()
- TfidfVectorizer()
- cosine_similarity()

*Exact Duplicates*
- DataFrame.duplicated()
- DataFrame.drop_duplicates()

*Record Linkage*
- recordlinkage.Index().block()
- recordlinkage.Index().SortedNeighbourhood()
- recordlinkage.Index().index()
- recordlinkage.Compare().compute()